<a href="https://colab.research.google.com/github/Ruijia0/SupplyChain_Analytics/blob/main/Parcel_Pickup_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from scipy.optimize import minimize
import folium
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

# Step 1: Fetch the Wikipedia data (manually extract or scrape)
data = {
    "Neighborhood": ["Buckhead", "Midtown", "Downtown", "Westside", "Old Fourth Ward", "Virginia-Highland", "Grant Park", "East Atlanta", "Brookhaven", "Candler Park"],
    "Population": [87000, 42000, 26000, 18000, 15000, 14000, 12000, 11000, 9500, 8700],
    "Latitude": [33.8486, 33.7814, 33.7550, 33.7845, 33.7594, 33.7765, 33.7330, 33.7428, 33.8604, 33.7652],
    "Longitude": [-84.3846, -84.3877, -84.3900, -84.4112, -84.3733, -84.3561, -84.3735, -84.3484, -84.3390, -84.3302]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Step 2: Define objective function for Haversine distance
def total_haversine_distance(coords):
    lat, lon = coords
    return sum(
        haversine(lat, lon, row.Latitude, row.Longitude) * row.Population
        for _, row in df.iterrows()
    )

# Minimize the total Haversine distance
initial_guess = [33.7490, -84.3880]  # Initial guess: Atlanta city center
result_haversine = minimize(
    total_haversine_distance, initial_guess, method="L-BFGS-B",
    bounds=[(33.7, 33.9), (-84.5, -84.3)]
)
optimal_haversine_location = result_haversine.x

# Step 3: Use driving distance API (mocked for simplicity)
def total_driving_distance(coords):
    lat, lon = coords
    # Mock driving distances using Haversine (replace with actual API calls if needed)
    return total_haversine_distance(coords)

result_driving = minimize(
    total_driving_distance, initial_guess, method="L-BFGS-B",
    bounds=[(33.7, 33.9), (-84.5, -84.3)]
)
optimal_driving_location = result_driving.x

# Step 4: Visualize results on a map
map = folium.Map(location=[33.7490, -84.3880], zoom_start=12)

# Add neighborhood markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row.Latitude, row.Longitude],
        popup=f"{row.Neighborhood}: {row.Population}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map)

# Add optimal locations
folium.Marker(
    location=optimal_haversine_location,
    popup="Optimal Haversine Location",
    icon=folium.Icon(color="green", icon="ok-sign")
).add_to(map)

folium.Marker(
    location=optimal_driving_location,
    popup="Optimal Driving Location",
    icon=folium.Icon(color="red", icon="ok-sign")
).add_to(map)

# Save map to an HTML file
map.save("optimal_parcel_pickup_locations.html")

# Output results
print("Optimal Haversine Location:", optimal_haversine_location)
print("Optimal Driving Location:", optimal_driving_location)

Optimal Haversine Location: [ 33.78174507 -84.38737123]
Optimal Driving Location: [ 33.78174507 -84.38737123]
